In [1]:
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
import pandas as pd
from scipy import stats
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from urllib.request import urlopen
import json
import dill

In [2]:
pre_2019=pd.read_csv('skater_stats.csv',encoding = "ISO-8859-1")

In [3]:
url='https://www.hockey-reference.com/leagues/NHL_2019_skaters.html'

In [4]:
r=requests.get(url)

In [5]:
soup = BeautifulSoup(r.content,'html.parser')

In [6]:
player_list=[]
for player in soup.find_all('tbody'):
    player_list.append(player.find_all('tr'))

In [7]:
player_list=player_list[0]

In [8]:
player_list[0]

<tr><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="left" csk="Abdelkader,Justin" data-append-csv="abdelju01" data-stat="player"><a href="/players/a/abdelju01.html">Justin Abdelkader</a></td><td class="right" data-stat="age">31</td><td class="left" data-stat="team_id"><a href="/teams/DET/2019.html">DET</a></td><td class="center" csk="1:Abdelkader:Justin" data-stat="pos">LW</td><td class="right" data-stat="games_played">71</td><td class="right" data-stat="goals">6</td><td class="right" data-stat="assists">13</td><td class="right" data-stat="points">19</td><td class="right" data-stat="plus_minus">-14</td><td class="right" data-stat="pen_min">38</td><td class="right" data-stat="ps">0.1</td><td class="right" data-stat="goals_ev">4</td><td class="right" data-stat="goals_pp">1</td><td class="right" data-stat="goals_sh">1</td><td class="right iz" data-stat="goals_gw">0</td><td class="right" data-stat="assists_ev">12</td><td class="right" data-stat="assists_pp">1</td>

In [9]:
len(player_list)

1121

In [10]:
a=[]
for stat in player_list[0].find_all('td'):
    a.append(stat.text)

In [11]:
b =[]
for i in range(0,1121):
    for stat in player_list[i].find_all('td'):
        b.append(stat.text)

In [12]:
b[0:27]

['Justin Abdelkader',
 '31',
 'DET',
 'LW',
 '71',
 '6',
 '13',
 '19',
 '-14',
 '38',
 '0.1',
 '4',
 '1',
 '1',
 '0',
 '12',
 '1',
 '0',
 '95',
 '6.3',
 '1093',
 '15:24',
 '34',
 '185',
 '52',
 '51',
 '50.5']

In [13]:
chunks = [b[x:x+27] for x in range(0, len(b), 27)]

In [14]:
chunks[0]

['Justin Abdelkader',
 '31',
 'DET',
 'LW',
 '71',
 '6',
 '13',
 '19',
 '-14',
 '38',
 '0.1',
 '4',
 '1',
 '1',
 '0',
 '12',
 '1',
 '0',
 '95',
 '6.3',
 '1093',
 '15:24',
 '34',
 '185',
 '52',
 '51',
 '50.5']

In [15]:
for stat in player_list[0].find_all(class_='right'):
    print(stat['data-stat'],stat.text)

ranker 1
age 31
games_played 71
goals 6
assists 13
points 19
plus_minus -14
pen_min 38
ps 0.1
goals_ev 4
goals_pp 1
goals_sh 1
goals_gw 0
assists_ev 12
assists_pp 1
assists_sh 0
shots 95
shot_pct 6.3
time_on_ice 1093
time_on_ice_avg 15:24
blocks 34
hits 185
faceoff_wins 52
faceoff_losses 51


In [16]:
df=pd.DataFrame(columns=['name','age','team','pos','gp','goals','assists','points','plusmin','plm','ps','goals_ev','goals_pp','goals_sh',
                        'goals_gw','assists_ev','assists_pp','assists_sh','shots','shot_pct','toi','toi_avg',
                        'hits','faceoff_wins','faceoff_losses','faceoff_percent'],
               index=range(0,1076))

In [17]:
df

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_pp,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
len(df.columns)

26

In [19]:
len(chunks)

1076

In [20]:
for i in range(len(chunks)):
    df.name[i] = chunks[i][0]

In [21]:
df.shape

(1076, 26)

In [22]:
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_pp,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent
0,Justin Abdelkader,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pontus Aberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pontus Aberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pontus Aberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vitaly Abramov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
chunks[0][22]

'34'

In [24]:
for i in range(len(df)):
    df.name[i] = chunks[i][0]
    df.age[i]=chunks[i][1]
    df.team[i]=chunks[i][2]
    df.pos[i]=chunks[i][3]
    df.gp[i]=chunks[i][4]
    df.goals[i]=chunks[i][5]
    df.assists[i]=chunks[i][6]
    df.points[i]=chunks[i][7]
    df.plusmin[i]=chunks[i][8]
    df.plm[i]=chunks[i][9]
    df.ps[i]=chunks[i][10]
    df.goals_ev[i]=chunks[i][11]
    df.goals_pp[i]=chunks[i][12]
    df.goals_sh[i]=chunks[i][13]
    df.goals_gw[i]=chunks[i][14]
    df.assists_ev[i]=chunks[i][15]
    df.assists_pp[i]=chunks[i][16]
    df.assists_sh[i]=chunks[i][17]
    df.shots[i]=chunks[i][18]
    df.shot_pct[i]=chunks[i][19]
    df.toi[i]=chunks[i][20]
    df.toi_avg[i]=chunks[i][21]
    df.hits[i]=chunks[i][23]
    df.faceoff_wins[i]=chunks[i][24]
    df.faceoff_losses[i]=chunks[i][25]
    df.faceoff_percent[i]=chunks[i][26]

In [25]:
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_pp,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,1,0,95,6.3,1093,15:24,185,52,51,50.5
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,4,0,101,11.9,861,14:36,45,2,17,10.5
2,Pontus Aberg,25,ANA,LW,37,11,8,19,-10,14,...,1,0,74,14.9,578,15:37,31,2,9,18.2
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,3,0,27,3.7,283,12:52,14,0,8,0.0
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,0,0,0,,14,13:52,0,0,0,


In [26]:
block_list = []
for i in range(len(chunks)):
    block_list.append(int(chunks[i][22]))

In [27]:
block_list[0:5]

[34, 11, 7, 4, 1]

In [28]:
len(block_list)

1076

In [29]:
df['block']=0
for i in range(len(df)):
    df.block[i]=block_list[i]

In [30]:
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent,block
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,0,95,6.3,1093,15:24,185,52,51,50.5,34
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,0,101,11.9,861,14:36,45,2,17,10.5,11
2,Pontus Aberg,25,ANA,LW,37,11,8,19,-10,14,...,0,74,14.9,578,15:37,31,2,9,18.2,7
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,0,27,3.7,283,12:52,14,0,8,0.0,4
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,0,0,,14,13:52,0,0,0,,1
